In [ ]:
from typing import Annotated
from typing_extensions import TypedDict

In [ ]:
from langchain_community.utilities import ArxivAPIWrapper,WikipediaAPIWrapper
from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun


arxiv_wrapper=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=300)
arxiv_tool=ArxivQueryRun(api_wrapper=arxiv_wrapper)

api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=300)
wiki_tool=WikipediaQueryRun(api_wrapper=api_wrapper)


In [ ]:
wiki_tool.invoke("who is elon musk?")

In [ ]:
arxiv_tool.invoke("attention is all you need")

In [ ]:
tools=[wiki_tool]

In [ ]:
from langgraph.graph.message import add_messages
class State(TypedDict):
    messages:Annotated[list,add_messages]

In [ ]:
from langchain_groq import ChatGroq

In [ ]:
##USING GROQ
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key=os.getenv("GROQ_API_KEY")
print(groq_api_key)

langsmith_api_key=os.getenv("LANGCHAIN_API_KEY")
print(langsmith_api_key)

In [ ]:
llm = ChatGroq(
    model="llama-3.1-8b-instant",
    groq_api_key=groq_api_key
)
llm

In [ ]:
llm_with_tools=llm.bind_tools(tools=tools)

In [ ]:
def chatbot(state:State):
    return {"messages":llm_with_tools.invoke(state['messages'])}

In [ ]:
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode, tools_condition

# Create graph
graph_builder = StateGraph(State)

# Chatbot node
graph_builder.add_node("chatbot", chatbot)

# Tool node
tool_node = ToolNode(tools=tools)
graph_builder.add_node("tools", tool_node)

# Entry point
graph_builder.add_edge(START, "chatbot")

# Conditional routing
graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition
)

# Tool → chatbot loop
graph_builder.add_edge("tools", "chatbot")

# Chatbot → END
graph_builder.add_edge("chatbot", END)

# Compile graph
graph = graph_builder.compile()


In [ ]:
from IPython.display import Image, display
try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    pass


In [ ]:
user_input="Hi there! ,my name is Varun"

events=graph.stream(
    {"messages":[("user",user_input)]},stream_mode="values"
)

for event in events:
    event["messages"][-1].pretty_print()